In [ ]:
# Librerías a usar
import dash
import dash_auth
from dash import dcc, html, Input, Output, State, callback_context
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import io
import flask
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Acceso con Usuario y contraseña
VALID_USERNAME_PASSWORD = {"admin": "1234"}

# asignación de clusters que vamos a usar
METAS_POR_CLUSTER = {
    "CRÍTICO": 95.0,    
    "MODERADO": 90.0,   
    "ESTABLE": 93.0,    
    "DESCONOCIDO": 98.0 
}

# Paleta colores a base de colores de la empresa
PALETA_KM = ["#868d8e", "#ec6c25", "#e7e5e5", "#e4b193", "#fcd2b9"]

COLORS = {
    "orange": "#ec6c25",      
    "green": "#28a745",        
    "yellow": "#ffc107",      
    "dark_blue": "#2c3e50", 
    "bg_light": "#f4f6f9",
    "text_grey": "#6c757d", 
    "card_grey": "#868d8e"
}

SIDEBAR_STYLE = {
    "position": "fixed", "top": 0, "left": 0, "bottom": 0,
    "width": "18rem", "padding": "2rem 1rem", "backgroundColor": "#f8f9fa",
    "overflowY": "auto", "zIndex": 1000
}

CONTENT_STYLE = {
    "marginLeft": "19rem", "marginRight": "2rem", "padding": "2rem 1rem",
    "backgroundColor": COLORS["bg_light"], "minHeight": "100vh"
}

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP], suppress_callback_exceptions=True)
app.title = "Logistica del Mayab AI Dashboard"
server = app.server
app.server.secret_key = 'super-secret-key'
auth = dash_auth.BasicAuth(app, VALID_USERNAME_PASSWORD)


#creación de página visual
def create_kpi_card(title, value, id_component=None, bg_color=COLORS["card_grey"]):
    return dbc.Card([
        dbc.CardBody([
            html.H6(title, className="card-title text-white small"),
            html.H3(value, id=id_component if id_component else "", className="card-text text-white fw-bold")
        ])
    ], style={"background-color": bg_color, "border": "none", "border-radius": "15px"}, className="shadow-sm h-100")

def fig_empty(title="Esperando datos..."):
    fig = go.Figure()
    fig.update_layout(title=title, xaxis={"visible": False}, yaxis={"visible": False}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', height=350)
    return fig

def upload_component(id_name, label_text):
    return html.Div([
        html.Label(label_text, className="fw-bold mb-2", style={"fontSize": "0.85rem"}),
        dcc.Upload(
            id=id_name,
            children=html.Div(['Arrastra o ', html.A('Selecciona Excel')]),
            style={
                'width': '100%', 'height': '50px', 'lineHeight': '50px',
                'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
                'textAlign': 'center', 'backgroundColor': 'white', 'cursor': 'pointer',
                'fontSize': '0.8rem'
            }, multiple=False
        ),
        html.Div(id=f"{id_name}-output", className="mt-1 small text-muted")
    ], className="mb-3")

# Organizacion de tabs y layout
layout_upload = html.Div([
    html.H2("Configuración de Datos", className="mb-4", style={"color": COLORS["dark_blue"], "fontWeight": "bold"}),
    html.P("Carga los archivos operativos para alimentar el modelo de IA y los KPIs.", className="text-muted mb-4"),
    
    dbc.Row([
        # KPI 1: COMBUSTIBLE
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("1. Combustible", className="text-white small", style={"backgroundColor": COLORS["dark_blue"]}),
                dbc.CardBody(upload_component('upload-combustible', 'Reporte Combustible (.xlsx)'))
            ], className="h-100 shadow-sm")
        ], md=4),
        
        # KPI 2: RUTAS 
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("2. Operaciones (Rutas & IA)", className="text-white small", style={"backgroundColor": COLORS["orange"]}),
                dbc.CardBody([
                    html.P("Este archivo alimenta el catálogo de rutas, el modelo de riesgo y la simulación.", className="small text-muted mb-2"),
                    upload_component('upload-viajes', 'Reporte Listado Viajes (.xlsx)'),
                    dbc.Row([
                        dbc.Col(html.Div(id='rutas-load-output', className="small fw-bold text-center"), width=6),
                        dbc.Col(html.Div(id='cluster-calc-output', className="small fw-bold text-center"), width=6)
                    ])
                ])
            ], className="h-100 shadow-sm")
        ], md=4),

        # KPI 3: KPIs FINANCIEROS
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("3. KPIs Financieros", className="text-white small", style={"backgroundColor": COLORS["dark_blue"]}),
                dbc.CardBody([
                    html.P("Calcula CPK y Costo Parado", className="small text-muted mb-2"),
                    upload_component('upload-telemetria', 'A. Telemetría'),
                    upload_component('upload-tablero', 'B. Tablero PL'),
                    html.Div(id='cpk-calc-output', className="text-center fw-bold mt-2 small"),
                    html.Div(id='ociosidad-calc-output', className="text-center fw-bold mt-1 small")
                ])
            ], className="h-100 shadow-sm")
        ], md=4),
    ]),

    dbc.Row([
        dbc.Col([
            html.Hr(),
            dbc.Button("Ir al Dashboard ->", href="/dashboard", style={"backgroundColor": COLORS["orange"], "border": "none"}, size="lg", className="mt-3")
        ], width=12, className="text-end")
    ])
])

layout_dashboard = html.Div([
    dbc.Row([dbc.Col(html.Div([html.I(className="bi bi-layout-sidebar me-3"), html.Span("Inicio / ", className="text-muted"), html.Span("Dashboard Operativo", className="fw-bold")], className="d-flex align-items-center mb-4"))]),
    
    dbc.Row([
        dbc.Col(create_kpi_card("Indicador Ociosidad", "---", id_component="kpi-ociosidad", bg_color=COLORS["orange"]), md=3),
        dbc.Col(create_kpi_card("Precio Diesel", "---", id_component="kpi-combustible"), md=3),
        dbc.Col(create_kpi_card("CPK Promedio", "---", id_component="kpi-cpk", bg_color=COLORS["orange"]), md=3),
        dbc.Col(create_kpi_card("Nivel de Servicio Meta", "98%", id_component="kpi-nivel-servicio"), md=3),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col([dbc.Card([dbc.CardBody([html.H6("ESTADO DE LA RUTA (CLÚSTER IA)", className="small text-muted mb-3 text-center"), html.Div([html.Div("---", id="cluster-circle-text", className="d-flex align-items-center justify-content-center fw-bold shadow-sm", style={"width": "120px", "height": "120px", "background-color": COLORS["card_grey"], "border-radius": "50%", "margin": "0 auto", "color": "white", "font-size": "1.1rem", "transition": "0.3s"})], className="text-center mb-4"), html.P("Estrategia Recomendada:", className="text-center text-muted small mb-1"), html.P("Seleccione una ruta o cargue datos.", id="cluster-strategy-text", className="text-center fw-bold small text-dark")])], className="h-100 shadow-sm border-0")], md=3),
        dbc.Col([dbc.Card(dbc.CardBody(dcc.Graph(id='grafico-tradeoff', figure=fig_empty("Esperando selección de ruta..."))), className="h-100 shadow-sm border-0")], md=9),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col([dbc.Card(dbc.CardBody(dcc.Graph(id='grafico-km', figure=fig_empty("Datos de KM pendientes..."))), className="shadow-sm border-0")], md=6),
        dbc.Col([dbc.Card(dbc.CardBody(dcc.Graph(id='grafico-buffer', figure=fig_empty("Cálculo de Buffer pendiente..."))), className="shadow-sm border-0")], md=6),
    ])
])



# Visual de sidebar y Uso de las search  
sidebar = html.Div([
    html.Div([html.Img(src="https://logimayab.com.mx/wp-content/uploads/2022/12/icono-logi-mayab.png", style={'height': '40px', 'width': 'auto', 'marginRight': '10px'}), html.H4("LOGI MAYAB", style={'display': 'inline-block', 'verticalAlign': 'middle'})], style={'marginBottom': '20px'}),
    html.Hr(),
    html.Div(id="user-display", className="mb-3 fw-bold text-center small"),
    dbc.Nav([dbc.NavLink([html.I(className="bi bi-upload me-2"), "Carga de Datos"], href="/", active="exact"), dbc.NavLink([html.I(className="bi bi-grid me-2"), "Dashboard"], href="/dashboard", active="exact")], vertical=True, pills=True, className="mb-4"),
    
    html.Div([
        html.Label("Filtro de Ruta:", className="text-muted small fw-bold mb-1"),
        dcc.Dropdown(
            id='sidebar-ruta-dropdown', 
            placeholder="Cargue rutas primero...",
            optionHeight=65,  
            style={
                'fontSize': '0.9rem', 
                'whiteSpace': 'normal',  
                'height': 'auto'        
            }
        )
    ], className="p-2 bg-white rounded border mt-auto")
], style=SIDEBAR_STYLE)

app.layout = html.Div([
    dcc.Location(id="url"),
    dcc.Store(id='store-combustible-price', storage_type='session'),
    dcc.Store(id='store-rutas-options', storage_type='session'),
    dcc.Store(id='store-telemetria-data', storage_type='session'),
    dcc.Store(id='store-tablero-data', storage_type='session'),
    dcc.Store(id='store-cpk-final', storage_type='session'),
    dcc.Store(id='store-cluster-results', storage_type='session'),
    dcc.Store(id='store-ociosidad-global', storage_type='session'),
    dcc.Store(id='store-viajes-raw', storage_type='session'),
    sidebar,
    html.Div(id="page-content", style=CONTENT_STYLE)
])

# Uso de callbacks para relación e interacción 
def find_period_for_date(date, periods_info):
    if pd.isna(date): return None
    matching_periods = periods_info[
        (periods_info['Fecha de inicio del periodo'] <= date) &
        (periods_info['Fecha de fin del periodo'] >= date)
    ]
    if not matching_periods.empty: return matching_periods['Periodo'].iloc[0]
    return None

@app.callback(Output("page-content", "children"), Output("user-display", "children"), [Input("url", "pathname")])
def render_page(pathname):
    username = flask.request.authorization['username'] if flask.request.authorization else "Admin"
    if pathname == "/dashboard": return layout_dashboard, f"Usuario: {username}"
    return layout_upload, f"Usuario: {username}"

@app.callback(Output('store-combustible-price', 'data'), Output('upload-combustible-output', 'children'), Input('upload-combustible', 'contents'))
def process_fuel(c):
    if not c: return None, ""
    try:
        df = pd.read_excel(io.BytesIO(base64.b64decode(c.split(',')[1])))
        if 'Precio Unitario Merc' in df.columns:
            df['Precio Unitario Merc'] = pd.to_numeric(df['Precio Unitario Merc'].astype(str).str.replace(r'[$,]', '', regex=True), errors='coerce')
            val = df[df['Precio Unitario Merc'] > 0]['Precio Unitario Merc'].mean()
            return val, html.Span(f"Precio: ${val:.2f}", className="text-success fw-bold")
    except: pass
    return None, html.Span(" Error archivo", className="text-danger")

@app.callback(Output('store-viajes-raw', 'data'), Input('upload-viajes', 'contents'))
def process_viajes_upload(c):
    if not c: return None
    return c 

@app.callback(Output('store-rutas-options', 'data'), Output('rutas-load-output', 'children'), Input('store-viajes-raw', 'data'))
def process_routes_from_store(c):
    if not c: return None, ""
    try:
        df = pd.read_excel(io.BytesIO(base64.b64decode(c.split(',')[1])))
        col = 'Ruta' if 'Ruta' in df.columns else df.columns[0]
        opts = [{'label': str(r), 'value': str(r)} for r in df[col].dropna().unique()]
        return opts, html.Span(f" {len(opts)} Rutas Cargadas", className="text-primary")
    except: return None, html.Span(" Error Rutas", className="text-danger")

@app.callback(Output('store-telemetria-data', 'data'), Output('upload-telemetria-output', 'children'), Input('upload-telemetria', 'contents'))
def process_telemetria(c):
    if not c: return None, ""
    return c, html.Span(" Listo", className="text-success")

@app.callback(Output('store-tablero-data', 'data'), Output('upload-tablero-output', 'children'), Input('upload-tablero', 'contents'))
def process_tablero(c):
    if not c: return None, ""
    return c, html.Span(" Listo", className="text-success")

# Naturaleza de los KPI con códigos
@app.callback(Output('store-cpk-final', 'data'), Output('cpk-calc-output', 'children'), Input('store-telemetria-data', 'data'), Input('store-tablero-data', 'data'))
def calc_cpk(tele_raw, tabl_raw):
    if not tele_raw or not tabl_raw: return None, ""
    try:
        df_tele = pd.read_excel(io.BytesIO(base64.b64decode(tele_raw.split(',')[1])))
        try: df_tele = pd.read_excel(io.BytesIO(base64.b64decode(tele_raw.split(',')[1])), sheet_name="Report")
        except: pass
        df_tele['Mes'] = pd.to_datetime(df_tele['Fecha de inicio del periodo']).dt.to_period('M').astype(str)
        dist_mes = df_tele.groupby('Mes')['Distancia'].sum().reset_index()

        df_pl = pd.read_excel(io.BytesIO(base64.b64decode(tabl_raw.split(',')[1])))
        try: df_pl = pd.read_excel(io.BytesIO(base64.b64decode(tabl_raw.split(',')[1])), sheet_name="Long PL Meses")
        except: pass
        df_pl = df_pl[df_pl["Rubro"] == "2. Costo"]
        df_pl['Mes'] = pd.to_datetime(df_pl['Date']).dt.to_period('M').astype(str)
        cost_mes = df_pl[["Mes", "Real"]]

        res = pd.DataFrame(cost_mes).merge(pd.DataFrame(dist_mes), on="Mes", how="left")
        cpk = (res["Real"] / res["Distancia"]).mean()
        return cpk, html.Div(f"CPK: ${cpk:.2f}", className="text-success")
    except: return None, html.Span("Error Calc CPK", className="text-danger")

@app.callback(Output('store-ociosidad-global', 'data'), Output('ociosidad-calc-output', 'children'), Input('store-tablero-data', 'data'), Input('store-telemetria-data', 'data'))
def calc_ociosidad_global(tablero_raw, telemetria_raw):
    if not tablero_raw or not telemetria_raw: return None, ""
    try:
        df_tele = pd.read_excel(io.BytesIO(base64.b64decode(telemetria_raw.split(',')[1])))
        try: df_tele = pd.read_excel(io.BytesIO(base64.b64decode(telemetria_raw.split(',')[1])), sheet_name="Report")
        except: pass
        unidades = len(df_tele["Nombre"].unique()) if "Nombre" in df_tele.columns else 1
        df_pl = pd.read_excel(io.BytesIO(base64.b64decode(tablero_raw.split(',')[1])))
        try: df_pl = pd.read_excel(io.BytesIO(base64.b64decode(tablero_raw.split(',')[1])), sheet_name="Long PL Meses")
        except: pass
        df_pl['Date'] = pd.to_datetime(df_pl['Date'])
        rubros = ['Salarios', 'Mantenimiento Automotriz', 'Depreciación', 'Arrendamiento']
        costo = df_pl[(df_pl['Rubro'].isin(rubros)) & (df_pl['Date'] == df_pl['Date'].max())]['Real'].sum()
        diario = (costo / unidades / 30) if unidades > 0 else 0
        return diario, html.Div(f"Ociosidad Global: ${diario:,.2f}/día", className="text-success")
    except: return None, html.Span("Error Ociosidad", className="text-danger small")

# Que clúster se usa por ruta
@app.callback(Output('store-cluster-results', 'data'), Output('cluster-calc-output', 'children'), Input('store-viajes-raw', 'data'))
def process_clustering(c):
    if not c: return None, ""
    try:
        df = pd.read_excel(io.BytesIO(base64.b64decode(c.split(',')[1])))
        df['Fecha Salida'] = pd.to_datetime(df['Fecha Salida'], errors='coerce')
        df['Fecha Llegada'] = pd.to_datetime(df['Fecha Llegada'], errors='coerce')
        df = df.dropna(subset=['Fecha Salida', 'Estatus de Viaje', 'Nro Ope', 'Operador', 'Tractocamión']).drop_duplicates(keep='first')
        mask = (df['Fecha Llegada'].dt.year <= 2025) & (df['Fecha Salida'].dt.year <= 2025) & (df['Fecha Salida'].dt.year >= 2024)
        df = df[mask]
        ruta_volumen = df.groupby('Ruta')['Viaje'].count().reset_index(name='Total_Viajes')
        ruta_cliente = df.groupby(['Ruta', 'Nombre Cliente'])['Viaje'].count().reset_index()
        ruta_max_cliente = ruta_cliente.sort_values(['Ruta', 'Viaje'], ascending=[True, False]).groupby('Ruta').first().reset_index()
        data_cluster = pd.merge(ruta_volumen, ruta_max_cliente[['Ruta', 'Viaje']], on='Ruta')
        data_cluster['Porcentaje_Dependencia'] = data_cluster['Viaje'] / data_cluster['Total_Viajes']
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(data_cluster[['Total_Viajes', 'Porcentaje_Dependencia']])
        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
        data_cluster['Cluster_Riesgo'] = kmeans.fit_predict(X_scaled)
        cluster_analysis = data_cluster.groupby('Cluster_Riesgo')['Porcentaje_Dependencia'].mean().sort_values(ascending=False)
        id_critico, id_estable = cluster_analysis.index[0], cluster_analysis.index[-1]
        id_moderado = [x for x in [0,1,2] if x not in [id_critico, id_estable]][0]
        mapa_estado = {int(id_critico): "CRÍTICO", int(id_moderado): "MODERADO", int(id_estable): "ESTABLE"}
        resultados = {}
        for idx, row in data_cluster.iterrows(): resultados[row['Ruta']] = mapa_estado.get(int(row['Cluster_Riesgo']), "DESCONOCIDO")
        return resultados, html.Span(" Modelo IA Entrenado", className="text-success")
    except: return None, html.Span(" Error ML", className="text-danger")

@app.callback(Output('kpi-combustible', 'children'), Input('store-combustible-price', 'data'))
def up_fuel(p): return f"${p:,.2f}" if p else "$0.00"

@app.callback(Output('kpi-cpk', 'children'), Input('store-cpk-final', 'data'))
def up_cpk(p): return f"${p:,.2f}" if p else "---"

@app.callback([Output('sidebar-ruta-dropdown', 'options'), Output('sidebar-ruta-dropdown', 'placeholder')], Input('store-rutas-options', 'data'))
def up_sidebar(o): return (o, "Seleccione ruta...") if o else ([], "Cargue archivo de Viajes...")

@app.callback(Output('kpi-nivel-servicio', 'children'), [Input('sidebar-ruta-dropdown', 'value'), Input('store-cluster-results', 'data')])
def update_service_level(ruta, cluster_data):
    if not ruta: return "98.0%"
    estado = cluster_data.get(ruta, "DESCONOCIDO") if cluster_data else "DESCONOCIDO"
    return f"{METAS_POR_CLUSTER.get(estado, 98.0)}%"

@app.callback([Output("cluster-circle-text", "children"), Output("cluster-circle-text", "style"), Output("cluster-strategy-text", "children")],
              [Input("sidebar-ruta-dropdown", "value"), Input("store-cluster-results", "data")])
def update_cluster_card(ruta, cluster_data):
    style = {"width": "120px", "height": "120px", "border-radius": "50%", "margin": "0 auto", "color": "white", "font-size": "1.1rem", "display": "flex", "align-items": "center", "justify-content": "center"}
    if not ruta or not cluster_data: 
        style["background-color"] = COLORS["card_grey"]
        return "---", style, "Datos pendientes..."
    estado = cluster_data.get(ruta, "N/A")
    color = COLORS["orange"] if estado == "CRÍTICO" else COLORS["yellow"] if estado == "MODERADO" else COLORS["green"] if estado == "ESTABLE" else COLORS["card_grey"]
    style.update({"background-color": color, "color": "black" if estado != "ESTABLE" else "white"})
    msg = " Diversificar" if estado == "CRÍTICO" else " Monitorear" if estado == "MODERADO" else " Mantener"
    return estado, style, msg

@app.callback(
    [Output('grafico-tradeoff', 'figure'), Output('kpi-ociosidad', 'children')],
    [Input('sidebar-ruta-dropdown', 'value'), Input('store-viajes-raw', 'data'), Input('store-cluster-results', 'data'), Input('store-ociosidad-global', 'data')]
)
def update_tradeoff_and_kpi(ruta_objetivo, viajes_contents, cluster_data, costo_global_val):
    fig_default = fig_empty("Esperando selección de ruta...")
    kpi_default = f"${costo_global_val:,.2f}/día" if costo_global_val else "---"
    if not ruta_objetivo or not viajes_contents: return fig_default, kpi_default

    try:
        estado = cluster_data.get(ruta_objetivo, "DESCONOCIDO") if cluster_data else "DESCONOCIDO"
        META_NS = METAS_POR_CLUSTER.get(estado, 98.0)
        df = pd.read_excel(io.BytesIO(base64.b64decode(viajes_contents.split(',')[1])))
        df['Fecha Salida'] = pd.to_datetime(df['Fecha Salida'], errors='coerce')
        df = df.dropna(subset=['Fecha Salida', 'Estatus de Viaje']).drop_duplicates()
        df_ruta = df[df['Ruta'] == ruta_objetivo].copy()
        
        if len(df_ruta) < 2: return fig_default, kpi_default

        demanda_semanal = df_ruta.set_index('Fecha Salida').resample('W')['Viaje'].count().fillna(0)
        capacidad_base = int(np.ceil(demanda_semanal.mean()))
        resultados_curva, punto_optimo = [], None

        for buffer in range(0, int(demanda_semanal.max()) + 5):
            capacidad_total = capacidad_base + buffer
            viajes_atendidos = np.minimum(demanda_semanal, capacidad_total)
            unidades_ociosas = np.maximum(0, capacidad_total - demanda_semanal)
            ns = (viajes_atendidos.sum() / demanda_semanal.sum() * 100) if demanda_semanal.sum() > 0 else 0
            cap_disp = capacidad_total * len(demanda_semanal)
            tasa_ocio = (unidades_ociosas.sum() / cap_disp * 100) if cap_disp > 0 else 0
            
            data_point = {'Flota_Total': capacidad_total, 'Nivel_Servicio': ns, 'Tasa_Ociosidad': tasa_ocio}
            resultados_curva.append(data_point)
            if punto_optimo is None and ns >= META_NS: punto_optimo = data_point

        df_curva = pd.DataFrame(resultados_curva)
        if punto_optimo is None: punto_optimo = resultados_curva[-1]

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_curva['Flota_Total'], y=df_curva['Nivel_Servicio'], name='Nivel Servicio', line=dict(color='#1f77b4', width=3)))
        fig.add_trace(go.Scatter(x=df_curva['Flota_Total'], y=df_curva['Tasa_Ociosidad'], name='Ociosidad', yaxis='y2', line=dict(color='#ff7f0e', dash='dot', width=3)))
        fig.add_shape(type="line", x0=df_curva['Flota_Total'].min(), x1=df_curva['Flota_Total'].max(), y0=META_NS, y1=META_NS, line=dict(color="green", dash="dash"), name=f"Meta {META_NS}%")
        fig.add_shape(type="line", x0=punto_optimo['Flota_Total'], x1=punto_optimo['Flota_Total'], y0=0, y1=100, line=dict(color="gray", width=1), yref='paper')
        
        fig.update_layout(title=f"Simulación: {ruta_objetivo}", xaxis_title="Tamaño de Flota", yaxis=dict(title="Nivel Servicio (%)", range=[60, 105]), yaxis2=dict(title="Ociosidad (%)", overlaying="y", side="right", range=[0, 100]), legend=dict(orientation="h", y=1.1), height=350, margin=dict(l=40, r=40, t=60, b=40))
        return fig, f"{punto_optimo['Tasa_Ociosidad']:.1f}% (Ruta)"
    except: return fig_default, kpi_default

# Gráfica de KM códificación
@app.callback(
    Output('grafico-km', 'figure'),
    [Input('sidebar-ruta-dropdown', 'value'), Input('store-viajes-raw', 'data'), Input('store-telemetria-data', 'data')]
)
def update_km_graph(ruta_objetivo, viajes_raw, telemetria_raw):
    fig_def = fig_empty("Datos de KM pendientes...")
    if not ruta_objetivo or not viajes_raw or not telemetria_raw: return fig_def

    try:
        df_viajes = pd.read_excel(io.BytesIO(base64.b64decode(viajes_raw.split(',')[1])))
        decoded_tele = base64.b64decode(telemetria_raw.split(',')[1])
        try: df_tele = pd.read_excel(io.BytesIO(decoded_tele), sheet_name="Report")
        except: df_tele = pd.read_excel(io.BytesIO(decoded_tele))
        
        df_tele = df_tele.drop_duplicates()
        km = df_tele.groupby(['Nombre', 'Periodo'])['Distancia'].sum().reset_index()
        periods_info = df_tele[['Periodo', 'Fecha de inicio del periodo', 'Fecha de fin del periodo']].drop_duplicates().sort_values('Fecha de inicio del periodo')

        df_viajes['Fecha Salida'] = pd.to_datetime(df_viajes['Fecha Salida'], errors='coerce')
        df_viajes = df_viajes.dropna(subset=['Fecha Salida', 'Nro Ope', 'Operador', 'Tractocamión'])
        
        route_trips = df_viajes[df_viajes['Ruta'] == ruta_objetivo].copy()
        if route_trips.empty: return fig_empty("Sin datos para esta ruta")

        route_trips['Matched_Periodo'] = route_trips['Fecha Salida'].apply(lambda x: find_period_for_date(x, periods_info))
        route_trips.dropna(subset=['Matched_Periodo'], inplace=True)
        route_trips['Matched_Periodo'] = route_trips['Matched_Periodo'].astype(int)

        route_trips_for_merge = route_trips[['Tractocamión', 'Matched_Periodo']].drop_duplicates().rename(columns={'Tractocamión': 'Nombre', 'Matched_Periodo': 'Periodo'})
        merged = pd.merge(route_trips_for_merge, km, on=['Nombre', 'Periodo'], how='left')
        merged.dropna(subset=['Distancia'], inplace=True)
        
        km_final = merged.groupby('Periodo')['Distancia'].sum().reset_index().sort_values('Periodo')

        fig = go.Figure(go.Bar(x=km_final['Periodo'], y=km_final['Distancia'], marker_color=[PALETA_KM[i % len(PALETA_KM)] for i in range(len(km_final))]))
        fig.update_layout(title=f"KM por Periodo: {ruta_objetivo}", xaxis_title="Periodo", yaxis_title="Distancia (km)", plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', height=300, margin=dict(l=40, r=20, t=40, b=40))
        return fig
    except Exception as e: return fig_empty(f"Error KM: {str(e)}")

# Códificación de gráfico de costos de buffer
@app.callback(
    Output('grafico-buffer', 'figure'),
    [Input('store-viajes-raw', 'data'),
     Input('store-ociosidad-global', 'data'),
     Input('store-cluster-results', 'data')]
)
def update_buffer_cost_graph(viajes_raw, costo_ociosidad_diario, cluster_results):
    fig_def = fig_empty("Simulación de Costos Pendiente...")
    if not viajes_raw or not costo_ociosidad_diario: return fig_def

    try:
        df = pd.read_excel(io.BytesIO(base64.b64decode(viajes_raw.split(',')[1])))
        df['Fecha Salida'] = pd.to_datetime(df['Fecha Salida'], errors='coerce')
        df = df.dropna(subset=['Fecha Salida', 'Estatus de Viaje']).drop_duplicates()
        top_rutas = df['Ruta'].value_counts().nlargest(30).index.tolist()
        resultados_simulacion = []
        
        for ruta in top_rutas:
            df_ruta = df[df['Ruta'] == ruta]
            demanda = df_ruta.set_index('Fecha Salida').resample('W')['Viaje'].count().fillna(0)
            if demanda.sum() == 0: continue
            
            estado = cluster_results.get(ruta, "DESCONOCIDO") if cluster_results else "DESCONOCIDO"
            meta_ns = METAS_POR_CLUSTER.get(estado, 98.0)
            capacidad_base = int(np.ceil(demanda.mean()))
            buffer_optimo = 0
            
            for buffer in range(0, max(1, int(demanda.max()) + 2)):
                capacidad = capacidad_base + buffer
                atendidos = np.minimum(demanda, capacidad)
                ns = (atendidos.sum() / demanda.sum() * 100)
                if ns >= meta_ns:
                    buffer_optimo = buffer
                    break
            
            costo_mensual = buffer_optimo * costo_ociosidad_diario * 30
            resultados_simulacion.append({'Ruta': ruta, 'Costo': costo_mensual, 'Estado': estado})
            
        df_res = pd.DataFrame(resultados_simulacion)
        df_res = df_res.sort_values('Costo', ascending=False).head(15)
        
        fig = px.bar(
            df_res, x='Costo', y='Ruta', orientation='h',
            title=f"Top 15 Rutas por Costo de Buffer (Ociosidad: ${costo_ociosidad_diario:,.0f}/día)",
            color='Estado',
            color_discrete_map={"CRÍTICO": COLORS["orange"], "MODERADO": COLORS["yellow"], "ESTABLE": COLORS["green"]},
            text_auto=',.0f'
        )
        fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', height=300, margin=dict(l=20, r=20, t=40, b=20), yaxis={'categoryorder':'total ascending'}, xaxis_title="Costo Mensual Estimado ($)")
        return fig
    except Exception as e: return fig_empty(f"Error Buffer Cost: {str(e)}")

if __name__ == "__main__":
    app.run(debug=True)